In [15]:
import numpy as np
import pandas as pd 
import keras
from keras.models import load_model
from keras.utils import np_utils
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.models import Model

## Method 1: load the complete h5 model to predict

In [2]:
model =None
model = load_model('/home/zxt/model_files/alexnet_s_40epoch.h5')

In [18]:
# model.summary()

In [3]:
X_test = np.load('/home/zxt/data/final_22/224/5k/X_test.npy')
Y_test = np.load('/home/zxt/data/final_22/224/5k/Y_test.npy')

In [4]:
X_test.shape

(11000, 224, 224, 3)

In [5]:
X_test /= 255.

In [6]:
Y_test.shape

(11000, 1)

In [7]:
for i in Y_test:
    if i[0]=='elephant':
        i[0]=0
    if i[0]=='wild_boar':
        i[0]=1
    if i[0]=='gazellethomosons':
        i[0]=2
    if i[0]=='squirrel':
        i[0]=3 
    if i[0]=='hedgehog':
        i[0]=4
    if i[0]=='guineafowl':
        i[0]=5
    if i[0]=='moose':
        i[0]=6
    if i[0]=='coyote':
        i[0]=7   
    if i[0]=='bird':
        i[0]=8
    if i[0]=='fox':
        i[0]=9
    if i[0]=='giraffe':
        i[0]=10
    if i[0]=='buffalo':
        i[0]=11
    if i[0]=='hare':
        i[0]=12
    if i[0]=='vehicle':
        i[0]=13
    if i[0]=='wildebeest':
        i[0]=14
    if i[0]=='cattle':
        i[0]=15 
    if i[0]=='skunk':
        i[0]=16
    if i[0]=='lion':
        i[0]=17
    if i[0]=='zebra':
        i[0]=18
    if i[0]=='black_bear':
        i[0]=19
    if i[0]=='racoon':
        i[0]=20
    if i[0]=='empty':
        i[0]=21

In [8]:
y_test=np_utils.to_categorical(Y_test,22)

In [9]:
loss, acc = model.evaluate(X_test, y_test)
print('restored model accuracy: {:5.2f}%'.format(acc*100))

11000/11000 [==============================] - 326s 30ms/step
restored model accuracy: 90.08%


##### notes: directly load the model could not correct predict 

## Method -2 : Load the model weights.ckpt

In [10]:
def alexnet_s(input_shape, n_classes):
    input = Input(input_shape)
  
    # actually batch normalization didn't exist back then
    # they used LRN (Local Response Normalization) for regularization
    x = Conv2D(96, 11, strides=4, padding='same', activation='relu')(input)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)
  
    x = Conv2D(256, 5, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)
  
    x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
    x = Conv2D(384, 3, strides=1, padding='same', activation='relu')(x)
  
    x = Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(3, strides=2)(x)
  
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)
    
    # S: add two more layers to train
    x = Dropout(0.4)(x)
    x = Dense(4096,activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(4096,activation='relu')(x)
    
    output = Dense(n_classes, activation='softmax')(x)
  
    model = Model(input, output)
    return model

In [11]:
input_shape = 224,224,3
n_classes=22


In [21]:
def create_model():
    model = alexnet_s(input_shape, n_classes)
    learning_rate = 0.01
    decay_rate = learning_rate /22
    momentum = 0.8
    sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
    return model


def get_model():
    global model
    model = create_model()
    model.load_weights('/home/zxt/model_files/alexnet_s_40epoch_weights.ckpt')
    print(" * Model loaded!")
    return model

In [22]:
a = get_model()

 * Model loaded!


In [26]:
loss, acc = a.evaluate(X_test,y_test)
print('restored model accuracy: {:5.2f}%'.format(acc*100))

11000/11000 [==============================] - 307s 28ms/step
restored model accuracy: 90.08%
